# Particles and Formulary

[astropy.units]: https://docs.astropy.org/en/stable/units/index.html
[plasmapy.particles]: https://docs.plasmapy.org/en/stable/particles/index.html
[plasmapy.formulary]: https://docs.plasmapy.org/en/stable/formulary/index.html
[decorators]: https://www.geeksforgeeks.org/decorators-in-python/
[Type hint annotations]: https://mypy.readthedocs.io/en/stable/cheat_sheet_py3.html

We'll start this notebook with a quick refresher on [astropy.units], before going through examples from [plasmapy.particles] ⚛️ and [plasmapy.formulary] 🧮. After an interlude about [type hint annotations] and [decorators], we'll discuss how to create a formulary function.

Let's start with some preliminary imports & settings. To execute a cell in a Jupyter notebook, press `Shift + Enter`. If you have to restart the notebook, please re-execute the following cell.

If using Google Colab, click **Run anyway** when prompted. (If prompted again, select **Restart runtime** when the installation finishes.)

In [ ]:
import sys

if 'google.colab' in str(get_ipython()):
    if 'plasmapy' not in sys.modules:
        !pip install plasmapy==2024.7.0

import astropy.units as u
import numpy as np
from plasmapy.particles import *
from plasmapy.particles.particle_class import valid_categories
from plasmapy.formulary import *
from plasmapy.utils.decorators import validate_quantities

## Quick refresher on `astropy.units`

[astropy.units]: https://docs.astropy.org/en/stable/units/index.html

We'll be using [astropy.units] throughout the rest of this notebook, so we'll begin with a brief reminder o fhow to use it. We typically import this subpackage as `u`.

We can create a physical quantity by multiplying or dividing a number or array with a unit.

In [ ]:
60 * u.km

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity

This operation creates a [Quantity] object: a number, sequence, or array that has been assigned a physical unit.  We can create [Quantity] objects with compound units.

In [ ]:
V = 88 * u.imperial.mile / u.hour
print(V)

[Quantity]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity
[Quantity.to()]: https://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity.to

We can use [Quantity.to()] to convert a [Quantity] to different units.

In [ ]:
V.to("m/s")

## Particles

[plasmapy.particles]: https://docs.plasmapy.org/en/stable/particles/index.html

The [plasmapy.particles] subpackage contains functions to access basic particle data and classes to represent particles.

In [ ]:
from plasmapy.particles import *

### Particle properties

[representation of a particle]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.ParticleLike.html#particlelike

There are several functions that provide information about different particles that might be present in a plasma. The input of these functions is a [representation of a particle], such as a string for the atomic symbol or the element name.

In [ ]:
atomic_number("Fe")

[atomic number]: https://en.wikipedia.org/wiki/Atomic_number

We can provide a number that represents the [atomic number].

In [ ]:
element_name(26)

We can provide standard symbols or the names of particles.

In [ ]:
is_stable("e-")

In [ ]:
charge_number("proton")

[mass number]: https://en.wikipedia.org/wiki/Mass_number
[Quantity]: https://docs.astropy.org/en/stable/units/quantity.html#quantity
[astropy.units]: https://docs.astropy.org/en/stable/units/index.html
[half_life]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.atomic.half_life.html#half-life

We can represent isotopes with the atomic symbol followed by a hyphen and the [mass number]. Let's use [half_life] to return the half-life of a radioactive particle in seconds as a [Quantity].

In [ ]:
half_life("C-14")

We typically represent an ion in a string by putting together the atomic symbol or isotope symbol, a space, the charge number, and the sign of the charge.

In [ ]:
charge_number("Fe-56 13+")

[particle-like]: https://docs.plasmapy.org/en/latest/glossary.html#term-particle-like
[plasmapy.particles]: https://docs.plasmapy.org/en/stable/particles/index.html

Functions in [plasmapy.particles] are quite flexible in terms of string inputs representing particles. An input is [particle-like] if it can be used to represent a physical particle.  

In [ ]:
particle_mass("iron-56 +13")

In [ ]:
particle_mass("iron-56+++++++++++++")

Most of these functions take additional arguments, with `Z` representing the charge number of an ion and `mass_numb` representing the mass number of an isotope. These arguments are often [keyword-only](https://docs.plasmapy.org/en/latest/glossary.html#term-keyword-only) to avoid ambiguity.

In [ ]:
particle_mass("Fe", Z=13, mass_numb=56)

### Particle objects

[Particle]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_class.Particle.html#plasmapy.particles.particle_class.Particle

Up until now, we have been using functions that accept representations of particles and then return particle properties. With the [Particle] class, we can create objects that represent physical particles.

In [ ]:
proton = Particle("p+")
electron = Particle("electron")
iron56_nucleus = Particle("Fe", Z=26, mass_numb=56)

[Particle]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_class.Particle.html#plasmapy.particles.particle_class.Particle

Particle properties can be accessed via attributes of the [Particle] class.

In [ ]:
proton.mass

In [ ]:
electron.charge

In [ ]:
electron.charge_number

In [ ]:
iron56_nucleus.nuclear_binding_energy

#### Antiparticles

[antiparticle]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_class.Particle.html#plasmapy.particles.particle_class.Particle.antiparticle
[Particle]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_class.Particle.html#plasmapy.particles.particle_class.Particle

We can get antiparticles of fundamental particles by using the [antiparticle] attribute of a [Particle].

In [ ]:
electron.antiparticle

[Particle]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_class.Particle.html#plasmapy.particles.particle_class.Particle


We can also use the tilde (`~`) operator on a [Particle] to get its antiparticle.

In [ ]:
~proton

#### Ionization and recombination

[Particle]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_class.Particle.html#plasmapy.particles.particle_class.Particle
[recombine()]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_class.Particle.html#plasmapy.particles.particle_class.Particle.recombine
[ionize()]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_class.Particle.html#plasmapy.particles.particle_class.Particle.ionize

The [recombine()] and [ionize()] methods of a [Particle] representing an ion or neutral atom will return a different [Particle] with fewer or more electrons.

In [ ]:
deuterium = Particle("D 0+")
deuterium.ionize()

When provided with a number, these methods tell how many bound electrons to add or remove.

In [ ]:
alpha = Particle("alpha")
alpha.recombine(2)

### Custom particles

[CustomParticle]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_class.CustomParticle.html

Sometimes we want to use a particle with custom properties.  For example, we might want to represent an average ion in a multi-species plasma or a dust particle.  For that we can use [CustomParticle].

In [ ]:
cp = CustomParticle(9e-26 * u.kg, 2.18e-18 * u.C, symbol="Fe 13.6+")

[CustomParticle]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_class.CustomParticle.html
[Particle]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_class.Particle.html

Many of the attributes of [CustomParticle] are the same as in [Particle].

In [ ]:
cp.mass

In [ ]:
cp.charge

In [ ]:
cp.symbol

[numpy.nan]: https://numpy.org/doc/stable/reference/constants.html#numpy.nan

If we do not include one of the physical quantities, it gets set to [numpy.nan] (not a number) in the appropriate units.

In [ ]:
CustomParticle(9.27e-26 * u.kg).charge

[CustomParticle]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_class.CustomParticle.html
[plasmapy.formulary]: https://docs.plasmapy.org/en/stable/formulary/index.html

[CustomParticle] objects can provided to most of the commonly used functions in [plasmapy.formulary], and we're planning to improve interoperability in future releases of PlasmaPy.

### Particle lists

[ParticleList]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_collections.ParticleList.html
[CustomParticle]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_class.CustomParticle.html
[Particle]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_class.Particle.html

The [ParticleList] class is a container for [Particle] and [CustomParticle] objects.

In [ ]:
iron_ions = ParticleList(["Fe 12+", "Fe 13+", "Fe 14+"])

[ParticleList]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_collections.ParticleList.html

By using a [ParticleList], we can access the properties of multiple particles at once.

In [ ]:
iron_ions.mass

In [ ]:
iron_ions.charge

In [ ]:
iron_ions.symbols

[ParticleList]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_collections.ParticleList.html
[CustomParticle]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_class.CustomParticle.html
[Particle]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_class.Particle.html

We can also create a [ParticleList] by adding [Particle] and/or [CustomParticle] objects together.

In [ ]:
proton + electron

We can also get an average particle.

In [ ]:
iron_ions.average_particle()

[ParticleList]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_collections.ParticleList.html
[plasmapy.formulary]: https://docs.plasmapy.org/en/stable/formulary/index.html

[ParticleList] objects can also be provided to the most commonly used functions in [plasmapy.formulary], with more complete interoperability expected in the future.

### Particle Categorization

[categories]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_class.Particle.html#plasmapy.particles.particle_class.Particle.categories
[Particle]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_class.Particle.html#particle
[set]: https://docs.python.org/3/library/stdtypes.html#set

The [categories] attribute of a [Particle] provides a set of the categories that the particle belongs to as a [set].

In [ ]:
muon = Particle("muon")
muon.categories

[is_category()]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_class.Particle.html#plasmapy.particles.particle_class.Particle.is_category
[Particle]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_class.Particle.html#particle

The [is_category()] method of a [Particle] lets us determine if the particle belongs to one or more categories.

In [ ]:
muon.is_category("lepton")

[Particle]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_class.Particle.html#particle

If we need to be more specific with `is_category()`, we can use its keyword arguments: 

 - `require`: for categories that a [Particle] _must_ belong to
 - `exclude`: for categories that the [Particle] _cannot_ belong to
 - `any_of`: for categories of which a [Particle] must belong to _at least one of_

In [ ]:
electron.is_category(require="lepton", exclude="baryon", any_of={"boson", "fermion"})

[plasmapy.particles.particle_class.valid_categories]: https://docs.plasmapy.org/en/latest/api/plasmapy.particles.particle_class.valid_categories.html

All valid particle categories are included in [plasmapy.particles.particle_class.valid_categories].

In [ ]:
print(valid_categories)

[ParticleList]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_collections.ParticleList.html#plasmapy.particles.particle_collections.ParticleList
[list]: https://docs.python.org/3/library/stdtypes.html#list

The `is_category()` method of [ParticleList] returns `True` if all particles match the criteria, and `False` otherwise.

In [ ]:
particles = ParticleList(["e-", "p+", "n"])
particles.is_category(require="lepton")

If we want to do this operation for all of the particles, we can set the `particlewise` argument to `True`. We will then get a `list` of booleans.

In [ ]:
particles.is_category(require="lepton", particlewise=True)

### Nuclear reactions

[plasmapy.particles]: https://docs.plasmapy.org/en/stable/particles/index.html

We can use [plasmapy.particles] to calculate the energy of a nuclear reaction using the `>` operator.  

In [ ]:
deuteron = Particle("D+")
triton = Particle("T+")
alpha = Particle("α")
neutron = Particle("n")

In [ ]:
energy = deuteron + triton > alpha + neutron

In [ ]:
energy.to("MeV")

If the nuclear reaction is invalid, then an exception is raised that states the reason why.

In [ ]:
deuteron + triton > alpha + 56 * neutron

## PlasmaPy formulary

[plasmapy.formulary]: https://docs.plasmapy.org/en/stable/formulary/index.html

The [plasmapy.formulary] subpackage contains a broad variety of formulas needed by plasma scientists across disciplines, in particular to calculate plasma parameters.

In [ ]:
from plasmapy.formulary import *

### Plasma beta in the solar corona

[Plasma beta]: https://en.wikipedia.org/wiki/Beta_(plasma_physics)

[Plasma beta] ($β$) is one of the most fundamental plasma parameters. $β$ is the ratio of the plasma (gas) pressure to the magnetic pressure. How a plasma behaves depends strongly on $β$. When $β ≫ 1$, the magnetic field is not strong enough to exert much of a force on the plasma, so its motions start to resemble a gas. When $β ≪ 1$, magnetic tension and pressure are the dominant macroscopic forces. 

Let's use [plasmapy.formulary](https://docs.plasmapy.org/en/stable/formulary/index.html) to calculate plasma β in different regions of the solar atmosphere and see what we can learn.

#### Solar corona

Let's start by defining some plasma parameters for an active region in the solar corona.

In [ ]:
B_corona = 50 * u.G
n_corona = 1e9 * u.cm ** -3
T_corona = 1 * u.MK

When we use these parameters in [beta](https://docs.plasmapy.org/en/stable/api/plasmapy.formulary.dimensionless.beta.html#plasmapy.formulary.dimensionless.beta), we find that $β$ is quite small so that the corona is magnetically dominated.

In [ ]:
beta(T=T_corona, n=n_corona, B=B_corona)

#### Solar photosphere

Let's specify some characteristic plasma parameters for the solar photosphere, away from any sunspots.

In [ ]:
T_photosphere = 5800 * u.K
B_photosphere = 400 * u.G
n_photosphere = 1e17 * u.cm ** -3

When we calculate for the photosphere, we find that it is an order of magnitude larger than 1, so plasma pressure forces are more important than magnetic tension and pressure.

In [ ]:
beta(T_photosphere, n_photosphere, B_photosphere)

### Plasma parameters in Earth's magnetosphere

[magnetic reconnection]: https://en.wikipedia.org/wiki/Magnetic_reconnection
[plasmapy.formulary]: https://docs.plasmapy.org/en/stable/formulary/index.html

The [*Magnetospheric Multiscale Mission*](https://www.nasa.gov/mission_pages/mms/overview/index.html) (*MMS*) is a constellation of four identical spacecraft. The goal of *MMS* is to investigate the small-scale physics of [magnetic reconnection] in Earth's magnetosphere. In order to do this, the spacecraft need to orbit in a tight configuration.  But how tight does the tetrahedron have to be?  Let's use [plasmapy.formulary] to find out.

#### Physics background

[Magnetic reconnection]: https://en.wikipedia.org/wiki/Magnetic_reconnection

[Magnetic reconnection] is the fundamental plasma process that converts stored magnetic energy into kinetic energy, thermal energy, and particle acceleration. Reconnection powers solar flares and is a key component of geomagnetic storms in Earth's magnetosphere. Reconnection can also degrade confinement in fusion devices such as tokamaks.

The **inertial length** is the characteristic length scale for a particle to get accelerated or decelerated by electromagnetic forces in a plasma.

When the reconnection layer thickness is shorter than the **ion inertial length**, $d_i ≡ c/ω_{pi}$, collisionless effects and the Hall effect enable reconnection to be fast (Zweibel & Yamada 2009). The inner electron diffusion region has a thickness of about the **electron inertial length**, $d_e ≡ c/ω_{pi}$. (Here, $ω_{pi}$ and $ω_{pe}$ are the ion and electron plasma frequencies.)

Our goal: calculate $d_i$ and $d_e$ to get an idea of how far the MMS spacecraft should be separated from each other to investigate reconnection.

### Length scales

Let's choose some characteristic plasma parameters for the magnetosphere.

In [ ]:
n = 1 * u.cm ** -3
B = 5 * u.nT
T = 30000 * u.K

Let's calculate the ion inertial length, $d_i$. On length scales shorter than $d_i$, the Hall effect becomes important as the ions and electrons decouple from each other.

In [ ]:
inertial_length(n=n, particle="p+").to("km")

The ion diffusion regions should therefore be a few hundred kilometers thick. Let's calculate the electron inertial length next.

In [ ]:
inertial_length(n=n, particle="e-").to("km")

The electron diffusion region should therefore have a characteristic length scale of a few kilometers, which is significantly smaller than the ion diffusion region.

We can also calculate the gyroradii for different particles 

In [ ]:
gyroradius(B=B, particle=["p+", "e-"], T=T).to("km")

The four *MMS* spacecraft have separations of ten to hundreds of kilometers, and thus are well-positioned to investigate Hall physics during reconnection in the magnetosphere.

#### Frequencies

We can calculate some of the fundamental frequencies associated with magnetospheric plasma. 

In [ ]:
plasma_frequency(n=n, particle=["p+", "e-"])

In [ ]:
gyrofrequency(B=B, particle=["p+", "e-"])

In [ ]:
lower_hybrid_frequency(B=B, n_i=n, ion="p+")

[lower_hybrid_frequency]: https://docs.plasmapy.org/en/stable/api/plasmapy.formulary.frequencies.lower_hybrid_frequency.html#lower-hybrid-frequency
[plasmapy.formulary]: https://docs.plasmapy.org/en/stable/formulary/index.html

Most of the functions in [plasmapy.formulary] have descriptions of the plasma parameters that include the formula and physical interpretation.  If we ever forget what the [lower_hybrid_frequency] represents, we can check out its documentation page!  

## Python interlude

[plasmapy.formulary]: https://docs.plasmapy.org/en/stable/formulary/index.html
[plasmapy.particles]: https://docs.plasmapy.org/en/stable/particles/index.html

Now that we've gone through how to use [plasmapy.particles] and [plasmapy.formulary], we'll move on to the process for adding a function to [plasmapy.formulary].  Before that, we need to introduce two capabilities for the Python language:

 - Type hint annotations
 - Decorators

### Type hint annotations

[_dynamically typed language_]: https://en.wikipedia.org/wiki/Dynamic_programming_language
[Type hint annotations]: https://mypy.readthedocs.io/en/stable/cheat_sheet_py3.html

In a _statically typed language_, variable types are explicitly declared. If `s` is declared to be a string, then `s` will always be a string. Type errors are found when code is _compiled_.

Python is a [_dynamically typed language_]. Variable types are determined at runtime rather than at compile time, and it's possible for variables to even change types!  

There are tradeoffs! ⚖️ Dynamically are _more flexible_, but at the cost of _reduced type safety_.

[Type hint annotations] provide a middle ground between statically vs. dynamically typed languages. 

🏷 Let's define a variable called `name` and provide it with a type hint annotation that says the variable should be a a `str`.

In [ ]:
name: str = "Darth Fortran"

The type hint of `str` in the above cell didn't actually do anything when we ran it. But when we read the code, it does tell us what type to expect `name` to be.

How about a `list` that starts empty but will eventually contain names?  

In [ ]:
names: list[str] = []

This type hint still doesn't do anything at runtime, but it helps us read the code, and lets us use code quality tools that can help us find errors.

We can specify that a variable should be one of multiple types with `|`:

In [ ]:
identifier: str | int

Type hints are particularly helpful when defining functions:

In [ ]:
def stringify(number: int) -> str:
    return str(num)

[static type checking]: https://mypy.readthedocs.io/en/stable/getting_started.html#dynamic-vs-static-typing
[mypy]: https://mypy.readthedocs.io/en/stable/

Many Python packages, including PlasmaPy, make use of [static type checking] tools like [mypy].  These tools can help us find type errors such as in the following function before we even run the code:

In [ ]:
def return_argument(x: int) -> str:
    return x

What if we want to specify that a function accepts a length and returns a volume?

In [ ]:
def volume(d: u.Quantity[u.m]) -> u.Quantity[u.m**3]:
    return d**3

### Decorators

In Python, functions are objects. This means that we can write a function that returns another function.

In [ ]:
def return_function():

    print("Defining inner_function...")
    
    def inner_function(): 
        print("Calling inner_function!")
    
    return inner_function

In [ ]:
function = return_function()

In [ ]:
function()

Or we can pass a function as an argument to another function!  We can use `typing.Callable` as the corresponding type hint annotation.

In [ ]:
from typing import Callable

In [ ]:
def apply_function(function: Callable, array):
    return function(array)

In [ ]:
array = [1, 2, 3, 4, 5]

In [ ]:
apply_function(max, array)

[**decorator**]: https://www.geeksforgeeks.org/decorators-in-python/

A function that _modifies another function_ is a [**decorator**]. 

Decorators in Python are a way to modify or enhance functions without changing their actual code. They wrap another function, potentially adding extra functionality before and after the original function runs.

In [ ]:
def decorator(function: Callable):
    
    def decorated_function() -> None:

        print("Before calling the function.")
        result = function()
        print("After calling the function.")

        return result

    return decorated_function

In [ ]:
def example_function():
    print("Inside original example_function!")

Let's try it out!

In [ ]:
modified_function = decorator(example_function)

In [ ]:
example_function()

In Python, we have _syntactic sugar_ for decorators, using `@`:

In [ ]:
@decorator
def function2():
    print("Inside function2!")

In [ ]:
function2()

#### Fibonacci sequence

Let's look at a _recursive_ function that computes the $n$th Fibonacci number.  If we define $F_0 ≡ 0$ and $F_1 ≡ 1$, then $ F_n = F_{n-1} + F_{n-2}$.

In [ ]:
def fibonacci(n: int) -> int:
    print("Calculating Fibonacci number for n =", n)
    return n if n < 2 else fibonacci(n - 1) + fibonacci(n - 2)

In [ ]:
fibonacci(2)

In [ ]:
fibonacci(4)

[@functools.cache]: https://docs.python.org/3/library/functools.html#functools.cache

We'll use [@functools.cache] to store the output of a function for a particular argument.

In [ ]:
from functools import cache


@cache
def fibonacci_cached(n: int) -> int:
    print("Calculating Fibonacci number for n =", n)
    return n if n < 2 else fibonacci_cached(n - 1) + fibonacci_cached(n - 2)

In [ ]:
fibonacci_cached(2)

In [ ]:
fibonacci_cached(5)

Let's try calling it again!

In [ ]:
fibonacci_cached(5)

## Writing a formulary function

[@particle_input]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.decorators.particle_input.html
[@validate_quantities]: https://docs.plasmapy.org/en/stable/api/plasmapy.utils.decorators.validators.validate_quantities.html#validate-quantities
[@check_relativistic]: https://docs.plasmapy.org/en/stable/api/plasmapy.utils.decorators.checks.check_relativistic.html#check-relativistic
[astropy.units]: https://docs.astropy.org/en/stable/units/index.html
[plasmapy.particles]: https://docs.plasmapy.org/en/stable/particles/index.html
[plasmapy.formulary]: https://docs.plasmapy.org/en/stable/formulary/index.html

PlasmaPy has three decorators that help us with writing formulary functions.

 - [@check_relativistic]
 - [@validate_quantities]
 - [@particle_input]

Let's get some imports out of the way.

In [ ]:
import astropy.units as u
from astropy import constants
from plasmapy.utils.decorators import check_relativistic
from plasmapy.particles import particle_input

### Checking relativistic velocities

[@check_relativistic]: https://docs.plasmapy.org/en/stable/api/plasmapy.utils.decorators.checks.check_relativistic.html#check-relativistic
[plasmapy.formulary]: https://docs.plasmapy.org/en/stable/formulary/index.html

Many functions that return velocities in [plasmapy.formulary] assume that relativistic velocities are unimportant. To check for this, we can use the [@check_relativistic] decorator.  

In [ ]:
@check_relativistic
def speed(distance, time):
    return distance / time

Let's try this with a velocity _approaching_ the speed of light.

In [ ]:
speed(299792457 * u.m, 1 * u.s)

Now let's try this with a velocity _exceeding_ the speed of light.

In [ ]:
speed(299792459 * u.m, 1 * u.s)

### Validating quantities

[Quantity]: https://docs.astropy.org/en/stable/units/quantity.html
[astropy.units]: https://docs.astropy.org/en/stable/units/
[plasmapy.formulary]: https://docs.plasmapy.org/en/stable/formulary/index.html

Most [plasmapy.formulary] functions accept and return [Quantity] objects from [astropy.units]. For example, we might write a function to calculate the magnetic pressure:

$$ p_B ≡ \frac{B^2}{2μ_o} $$

In [ ]:
def magnetic_pressure(B):
    return B**2 / (2 * constants.mu0)

In [ ]:
magnetic_pressure(1 * u.T)

But the above function doesn't prevent us from making mistakes simple mistakes...

In [ ]:
magnetic_pressure(1 * u.B)

[@validate_quantities]: https://docs.plasmapy.org/en/latest/api/plasmapy.utils.decorators.validators.validate_quantities.html#validate-quantities

The [@validate_quantities] decorator makes sure that  

In [ ]:
@validate_quantities
def magnetic_pressure(B: u.Quantity[u.T]) -> u.Quantity[u.Pa]:
    return B**2 / (2 * constants.mu0)

In [ ]:
magnetic_pressure(1 * u.T)

In [ ]:
magnetic_pressure(1 * u.B)

In [ ]:
magnetic_pressure(1)

[@validate_quantities]: https://docs.plasmapy.org/en/latest/api/plasmapy.utils.decorators.validators.validate_quantities.html#validate-quantities
[equivalency]: https://docs.astropy.org/en/stable/units/equivalencies.html
[astropy.units]: https://docs.astropy.org/en/stable/units/


We can also provide arguments directly to [@validate_quantities], for example if we want to do a validation on the return value whilst using an [equivalency] from [astropy.units].

In [ ]:
@validate_quantities(
    validations_on_return={"units": u.K, "equivalencies": u.temperature_energy()}
)
def get_temperature(T):
    return T

In [ ]:
get_temperature(1 * u.eV)

The docstring for [@validate_quantities](https://docs.plasmapy.org/en/latest/api/plasmapy.utils.decorators.validators.validate_quantities.html#validate-quantities) contains examples for several more validations that it can do, such as:

 - Allowing only non-negative values
 - Restricting/allowing `complex` values
 - Restricting/allowing `numpy.nan` values 
 - Allowing `None` instead of a `Quantity`

## Particle inputs

[particle-like]: https://docs.plasmapy.org/en/latest/glossary.html#term-particle-like

Plasma parameters often depend on particle properties, especially the particle's charge and mass. 

Early on, we wrote formulary functions like this:

In [ ]:
def get_particle_mass(particle_string):
    particle_object = Particle(particle_string)
    return particle_object.mass

In [ ]:
get_particle_mass("p+")

[ParticleList]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_collections.ParticleList.html

The above example isn't particularly bad. But if we wanted to make a function compatible with [ParticleList] objects too, then it started to become a mess.

In [ ]:
from collections.abc import Iterable


def get_particle_mass(particle):
    if isinstance(particle, str):
        particle_object = Particle(particle)
    elif isinstance(particle, Iterable):
        particle_object = ParticleList(particle)
    return particle_object.mass

In [ ]:
get_particle_mass(["p+", "e-"])

[**@particle_input**]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.decorators.particle_input.html
[ParticleLike]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_class.ParticleLike.html#particlelike
[ParticleList]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_collections.ParticleList.html
[CustomParticle]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_class.CustomParticle.html
[Particle]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_class.Particle.html


The [**@particle_input**] decorator transforms arguments annotated with [ParticleLike] into a [Particle], [CustomParticle], or [ParticleList]. This transformation occurs _before_ the argument gets passed into the decorated function, so we end up with cleaner code!

In [ ]:
@particle_input
def make_particle(particle: ParticleLike):
    return particle

Let's try passing it a string representing a particle.

In [ ]:
make_particle("p+")

[Particle]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.particle_class.Particle.html
[@particle_input]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.decorators.particle_input.html


The argument got converted into a [Particle] object by [@particle_input].

If we add `Z` and `mass_numb` as parameters, they'll get incorporated into the particle.

In [ ]:
@particle_input
def make_particle(particle: ParticleLike, Z=None, mass_numb=None):
    return particle

In [ ]:
make_particle("He", Z=2, mass_numb=4)

[@particle_input]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.decorators.particle_input.html

If we name the parameter `element`, `ion`, or `isotope`, then [@particle_input] will make sure that the particle matches that identity. 

In [ ]:
@particle_input
def mass_number(isotope: ParticleLike):
    return isotope.mass_number

In [ ]:
mass_number("Fe-56")

In [ ]:
mass_number("e-")

### Putting it all together

[@validate_quantities]: https://docs.plasmapy.org/en/latest/api/plasmapy.utils.decorators.validators.validate_quantities.html#validate-quantities
[@particle_input]: https://docs.plasmapy.org/en/stable/api/plasmapy.particles.decorators.particle_input.html
[@check_relativistic]: https://docs.plasmapy.org/en/stable/api/plasmapy.utils.decorators.checks.check_relativistic.html#check-relativistic

Let's use both [@particle_input], [@validate_quantities], and [@check_relativistic] to write an Alfvén speed function. The formula is $$ V_A ≡ \frac{B}{\sqrt{μ_0 ρ}} $$ where we approximate the mass density as $ρ ≈ m_i n_i$.  

In [ ]:
@particle_input
@check_relativistic
@validate_quantities
def alfven_speed(
    B: u.Quantity[u.T],
    n: u.Quantity[u.m**-3], 
    ion: ParticleLike,
) -> u.Quantity[u.m / u.s]:
    """Return the Alfvén speed."""
    return B / np.sqrt(constants.mu0 * n * ion.mass)

In [ ]:
alfven_speed(B = 0.02 * u.T, n = 1e15 * u.m**-3, ion="p+")